### T1: Limpeza dos csv brutos do Siconfi

In [ ]:
import pandas as pd
import sys
from pathlib import Path

# Adiciona a raiz do projeto para importar o config
sys.path.append(str(Path.cwd().parent))
from src import config as cfg

# Caminhos baseados no config.py
input_dir = cfg.RAW_DIR / 'siconfi' / 't0'
output_dir = cfg.RAW_DIR / 'siconfi' / 't1'

# Cria a pasta de destino caso não exista
output_dir.mkdir(parents=True, exist_ok=True)

# Lista todos os arquivos CSV usando pathlib
files = list(input_dir.glob('*.csv'))

print(f"Iniciando processamento de {len(files)} arquivos...")

for file_path in files:
    # O nome de saída é o próprio nome do arquivo de entrada
    output_path = output_dir / file_path.name
    
    try:
        # Lemos pulando as 3 linhas e definindo o encoding original (latin1)
        df = pd.read_csv(file_path, skiprows=3, sep=';', encoding='latin1')
        
        # Salvamos no destino com encoding utf-8
        df.to_csv(output_path, index=False, encoding='utf-8', sep=';')
        
        print(f"✅ Sucesso: {file_path.name}")
        
    except Exception as e:
        print(f"❌ Erro ao processar {file_path.name}: {e}")

print("\nProcessamento concluído!")

Iniciando processamento de 13 arquivos...
✅ Sucesso: finbra_mun_despesas-por-funcao_2013.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2014.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2015.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2016.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2017.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2018.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2019.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2020.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2021.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2022.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2023.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2024.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2025.csv

Processamento concluído!


### T2: Filtrar apenas as contas desejadas ('17 - Saneamento' e '18 - Gestão Ambiental')

In [ ]:
import pandas as pd
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))
from src import config as cfg

# Caminhos da t1 para a t2
input_dir = cfg.RAW_DIR / 'siconfi' / 't1'
output_dir = cfg.RAW_DIR / 'siconfi' / 't2'
output_dir.mkdir(parents=True, exist_ok=True)

# Contas alvo
contas_alvo = [
    "17 - Saneamento",
    "18 - Gestão Ambiental"
]

files = list(input_dir.glob('*.csv'))
print(f"Iniciando a filtragem de {len(files)} arquivos...")

for file_path in files:
    output_path = output_dir / file_path.name
    
    try:
        # Lendo o arquivo já limpo (UTF-8)
        df = pd.read_csv(file_path, sep=';', encoding='utf-8')
        
        # Aplicando o filtro
        df_filtrado = df[df['Conta'].isin(contas_alvo)]
        
        # Salvando
        df_filtrado.to_csv(output_path, index=False, sep=';', encoding='utf-8')
        
        print(f"✔️ Filtrado: {file_path.name} ({len(df_filtrado)} linhas encontradas)")
        
    except Exception as e:
        print(f"❌ Erro ao filtrar {file_path.name}: {e}")

print("\nFiltragem concluída com sucesso!")

Iniciando a filtragem de 13 arquivos...
✔️ Filtrado: finbra_mun_despesas-por-funcao_2013.csv (25164 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2014.csv (25059 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2015.csv (26411 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2016.csv (24796 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2017.csv (26410 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2018.csv (27600 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2019.csv (27820 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2020.csv (26924 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2021.csv (27948 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2022.csv (29326 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2023.csv (30095 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2024.csv (29080 linhas encontradas)


### T3: Padronização e calculo do valor per capita
- Remover as colunas 'Instituição' e 'Identificador da Conta'
- Alterar o nome da coluna 'Cod.IBGE' para 'cd_mun'
- Padronizar o nome de todas as colunas, passando elas para minúsculo, escreva 'População' como 'populacao' e coloque underlines em espaçoes
- Calcular o valor per capita (utilizando a coluna 'Valor' e 'População') e salve na coluna 'valor_per_capita'
- Selecionar apenas as 'Despesas Liquidadas'

In [ ]:
import pandas as pd
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))
from src import config as cfg

# Caminhos da t2 para a t3
input_dir = cfg.RAW_DIR / 'siconfi' / 't2'
output_dir = cfg.RAW_DIR / 'siconfi' / 't3'
output_dir.mkdir(parents=True, exist_ok=True)

def padronizar_colunas(col):
    col = str(col).lower()
    col = col.replace('ç', 'c').replace('ã', 'a').replace('é', 'e').replace('õ', 'o')
    col = col.replace(' ', '_')
    return col

files = list(input_dir.glob('*.csv'))

for file_path in files:
    output_path = output_dir / file_path.name
    
    try:
        # decimal=',' resolve a leitura do Siconfi
        df = pd.read_csv(file_path, sep=';', encoding='utf-8', decimal=',')
        
        # 1. Padronizar nomes de colunas
        df.columns = [padronizar_colunas(c) for c in df.columns]
        
        # 2. Filtrar apenas Despesas Liquidadas
        if 'coluna' in df.columns:
            df = df[df['coluna'] == 'Despesas Liquidadas']
        
        # 3. Renomear e remover colunas irrelevantes
        df = df.rename(columns={'cod.ibge': 'cd_mun'})
        cols_para_remover = ['instituicao', 'identificador_da_conta']
        df = df.drop(columns=[c for c in cols_para_remover if c in df.columns], errors='ignore')

        # 4. Tratar colunas numéricas
        for col in ['valor', 'populacao']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        
        # 5. Cálculo Per Capita
        if 'valor' in df.columns and 'populacao' in df.columns:
            df['valor_per_capita'] = 0.0
            mask = df['populacao'] > 0
            df.loc[mask, 'valor_per_capita'] = df.loc[mask, 'valor'] / df.loc[mask, 'populacao']
            
            # Salvar o arquivo final da T3
            df.to_csv(output_path, index=False, sep=';', encoding='utf-8')
            print(f"✅ Processado: {file_path.name} | Linhas: {len(df)}")
        else:
            print(f"⚠️ Colunas não encontradas em {file_path.name}: {df.columns.tolist()}")

    except Exception as e:
        print(f"❌ Erro em {file_path.name}: {e}")

print("\n--- Processo finalizado! O script g1_siconfi.py já pode consumir a pasta T3. ---")

✅ Processado: finbra_mun_despesas-por-funcao_2013.csv | Linhas: 6567
✅ Processado: finbra_mun_despesas-por-funcao_2014.csv | Linhas: 6434
✅ Processado: finbra_mun_despesas-por-funcao_2015.csv | Linhas: 6717
✅ Processado: finbra_mun_despesas-por-funcao_2016.csv | Linhas: 6661
✅ Processado: finbra_mun_despesas-por-funcao_2017.csv | Linhas: 6791
✅ Processado: finbra_mun_despesas-por-funcao_2018.csv | Linhas: 6995
✅ Processado: finbra_mun_despesas-por-funcao_2019.csv | Linhas: 7134
✅ Processado: finbra_mun_despesas-por-funcao_2020.csv | Linhas: 7277
✅ Processado: finbra_mun_despesas-por-funcao_2021.csv | Linhas: 7287
✅ Processado: finbra_mun_despesas-por-funcao_2022.csv | Linhas: 7607
✅ Processado: finbra_mun_despesas-por-funcao_2023.csv | Linhas: 7561
✅ Processado: finbra_mun_despesas-por-funcao_2024.csv | Linhas: 7577
✅ Processado: finbra_mun_despesas-por-funcao_2025.csv | Linhas: 419

--- Processo finalizado! Verifique os valores agora. ---
